In [2]:
import sys
sys.executable


'c:\\ProgramData\\anaconda3\\envs\\machinelearning\\python.exe'

In [3]:
import os
import sys

# Add project root to Python path
sys.path.append(os.path.abspath(".."))

print("Project root added to path")


Project root added to path


In [4]:
!pip install PyPDF2 docx2txt
!pip install spacy
!python -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 2.2 MB/s eta 0:00:06
     ---- ----------------------------------- 1.3/12.8 MB 2.2 MB/s eta 0:00:06
     ----- ---------------------------------- 1.8/12.8 MB 2.3 MB/s eta 0:00:05
     -------- ------------------------------- 2.6/12.8 MB 2.6 MB/s eta 0:00:04
     ---------- ----------------------------- 3.4/12.8 MB 2.9 MB/s eta 0:00:04
     ------------- -------------------------- 4.2/12.8 MB 3.0 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 2.9 MB/s eta 0:00:03
     ---------------- ----------------------- 5.2/12.8 MB 2.9 MB/s et

In [5]:
from parser.resume_parser import parse_resume
from nlp.text_preprocessing import clean_text

In [6]:
raw_text = parse_resume("../uploads/sample_resume.pdf")
cleaned_text = clean_text(raw_text)

print(cleaned_text[:500])


harshit tiwari 
   harshittiwari         linkedinharshit tiwari         
 education srm institute science technology kattankulathur expect     
 bachelor technology computer science specialization big datum analytic cumulative gpa         
 technical skill programming language python web html css framework django library panda numpy scikit learn matplotlib seaborn database mysql sqlite tool git github google colab code jupyter notebook excel project house price prediction github build model pred


In [7]:
print("RAW TEXT LENGTH:", len(raw_text))
print("CLEANED TEXT LENGTH:", len(cleaned_text))


RAW TEXT LENGTH: 1712
CLEANED TEXT LENGTH: 1420


In [8]:
from nlp.skill_extraction import extract_skills
resume_skills = extract_skills(cleaned_text)
print("Extracted Skills:", resume_skills)

Extracted Skills: ['numpy', 'mysql', 'django', 'css', 'sql', 'scikit', 'html', 'python']


In [9]:
job_description = """We are looking for a Python Django developer with strong SQL skills.
Experience with REST APIs and AWS is preferred.
Knowledge of HTML, CSS, and JavaScript is a plus."""

CLEANING THE JOB DESCRIPTION

In [10]:
cleaned_jd= clean_text(job_description)
print("CLEANED JOB DESCRIPTION", cleaned_jd)

CLEANED JOB DESCRIPTION look python django developer strong sql skill experience rest apis aw preferred knowledge html css javascript plus


In [11]:
jd_skills = extract_skills(cleaned_jd)
print("Extracted JD Skills:", jd_skills)

Extracted JD Skills: ['java', 'javascript', 'django', 'rest', 'css', 'sql', 'api', 'html', 'python']


CREATING COMMON SKILLS BETWEEN JOB DESCRIPTION AND RESUME AND CALCULATING ITS OVERLAP SCORE

In [12]:
common_skills = set(resume_skills).intersection(set(jd_skills))

skill_overlap_score = len(common_skills) / len(jd_skills)

print("Common Skills:", common_skills)
print("Skill Overlap Score:", skill_overlap_score)

Common Skills: {'django', 'css', 'sql', 'html', 'python'}
Skill Overlap Score: 0.5555555555555556


# Load Dataset Resumes

In [13]:
import json

resumes = []

with open("../data/resumes_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line.strip())
        resumes.append(data["Text"])

print("Total resumes loaded:", len(resumes))


Total resumes loaded: 3500


**CLEANING THE LOADED RESUMES TEXT USING OUR FUNCTION OR THE PIPELINE WE MADE **

In [14]:
cleaned_resumes = []

for r in resumes:
    cleaned = clean_text(r)
    cleaned_resumes.append(cleaned)

print("Total cleaned resumes:", len(cleaned_resumes))


Total cleaned resumes: 3500


In [15]:
job_description = """
We are looking for a Python Django developer with strong SQL skills.
Experience with REST APIs and AWS is preferred.
Knowledge of HTML, CSS, and JavaScript is a plus.
"""


In [16]:
cleaned_jd = clean_text(job_description)

print(cleaned_jd)


look python django developer strong sql skill experience rest apis aw preferred knowledge html css javascript plus


In [17]:
%pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create vectorizer
vectorizer = TfidfVectorizer()

# Combine JD and resumes
documents = [cleaned_jd] + cleaned_resumes

# Fit and transform
tfidf_matrix = vectorizer.fit_transform(documents)

print("TF-IDF matrix shape:", tfidf_matrix.shape)


TF-IDF matrix shape: (3501, 38235)


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
# Job description vector
jd_vector=tfidf_matrix[0:1]
# Resume vectors
resume_vectors=tfidf_matrix[1:]
# Compute similarity
similarity_scores = cosine_similarity(jd_vector, resume_vectors).flatten()

print("Computed similarity for", len(similarity_scores), "resumes")

Computed similarity for 3500 resumes


In [20]:
print(similarity_scores[:10])


[0.02124501 0.01867914 0.0797577  0.09754874 0.03935882 0.03947394
 0.11927576 0.03479041 0.07400253 0.06189171]


In [ ]:
#Similarity score → semantic relevance
#Skill score → requirement match


from nlp.skill_extraction import extract_skills

# Extract JD skills
jd_skills = extract_skills(cleaned_jd)

resume_skill_scores = []

for r in cleaned_resumes:
    skills = extract_skills(r)
    
    overlap = set(skills).intersection(set(jd_skills))
    
    score = len(overlap) / len(jd_skills) if jd_skills else 0
    
    resume_skill_scores.append(score)

print("Computed skill scores for", len(resume_skill_scores), "resumes")


Computed skill scores for 3500 resumes


Calculating Total score   giving more priority to similarity score
Similarity score → semantic relevance         
Skill score → requirement match

In [22]:
final_scores = []

for sim, skill in zip(similarity_scores, resume_skill_scores):
    final_scores.append(0.7 * sim + 0.3 * skill)

print("Final scores computed.")


Final scores computed.


In [23]:
import numpy as np

ranked_indices = np.argsort(final_scores)[::-1]

print("Top 5 candidates scores:")

for i in ranked_indices[:5]:
    print(final_scores[i])


Top 5 candidates scores:
0.5130227262521225
0.5000167994152296
0.4876267235485146
0.4736196537890567
0.46508569995179533


In [24]:
top_idx = ranked_indices[0]

print("Top candidate similarity:", similarity_scores[top_idx])
print("Top candidate skill score:", resume_skill_scores[top_idx])
print("Top candidate final score:", final_scores[top_idx])

print("\nResume snippet:\n")
print(cleaned_resumes[top_idx][:500])


Top candidate similarity: 0.3043181803601751
Top candidate skill score: 1.0
Top candidate final score: 0.5130227262521225

Resume snippet:

jessica claire      montgomery floor              resumesampleexamplecom professional summary year experience analysis design development management implementation standalone clientserver architecture base enterprise application software extensive knowledge python mysql django experience design code debug operation report datum analysis web application utilize python work mvw framework like django angular html css json excellent working experience agile scrum rally tfs rup waterfall methodology 
